In [1]:
from helper import *
from datasetcreator import *
from dependencies import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata

Using TensorFlow backend.


User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [2]:
cv, bid_2_stars_ne, review_data_ne, bid_2_textenc = give_text_feats_tfidf(review_data)
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()

Size of Vocabulary:  15374


In [303]:
def give_reg_model_rf(n=100,mins = 1):
    print("Random Forest")
    model = RandomForestRegressor(n_estimators = n,oob_score = True, min_samples_leaf  = mins, random_state = 2, n_jobs=-1)
    return model

In [300]:
xfeat,yfeat= create_dataset_everything(40,business_data,bid_2_stars_ne,bid_2_stars_e,bid_2_textenc)
xtrain, xtest, ytrain, ytest = give_train_test(xfeat, yfeat)
model = give_reg_model_rf(n=2000, mins=30)
model, rsqr  = test_model(model,xtrain,ytrain,xtest,ytest)

Dimension of Our Dataset:  (5926, 47)
Random Forest
MSE:  0.0398
R-Squared :  0.774
Margin  0.10  0.20  0.30  0.40  0.50
        40.39  71.42  87.77  95.03  97.81
 


In [301]:
from sklearn.tree import export_graphviz

In [298]:
import pydot
tree = model.estimators_[0]
f1 = ['NE-Avg','%1 star','%2 star','%3 star','%4 star','%5 star']
f2 = f1 + ['' for x in range(0,41)]
export_graphviz(tree, out_file = 'tree.dot',feature_names = f2,rounded = True, precision = 3)


In [299]:
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree2.png')

In [327]:
def create_dataset_everything_text(min_thold, bdata, bid_2_stars, bid_2_stars_e, bid_2_textenc, max_thold=float('inf')):
    basicprint = False
    x1 = []
    x2 = []
    y = []
    bids = bdata['business_id'].values
    for bid in bids:
        if bid in bid_2_stars and bid in bid_2_stars_e and bid in bid_2_textenc:
            stars_e = bid_2_stars_e[bid]
            stars_ne = bid_2_stars[bid]
            lestars = len(stars_e)
            if(lestars >= min_thold and lestars < max_thold):
                x = []
                x.append([np.mean(stars_ne)])
                countratings = [Counter(stars_ne)[i] for i in range(1,6)]
                distribution = [x/sum(countratings) for x in countratings]
                x.append(distribution)
                x.append(give_categ(np.mean(stars_ne)))
                x.append(bid_2_textenc[bid])
                x = [item for sublist in x for item in sublist]
                x1.append(x)
                y.append(np.mean(stars_e))                        
    xfeat = np.array(x1)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    xfeat = imp.fit_transform(xfeat)
    yfeat = np.array(y).reshape(-1,)
    print("Dimension of Our Dataset: ", xfeat.shape)
    return xfeat,yfeat
  

In [328]:
xfeat,yfeat= create_dataset_everything_text(40,business_data,bid_2_stars_ne,bid_2_stars_e,bid_2_textenc)
xtrain, xtest, ytrain, ytest = give_train_test(xfeat, yfeat)
model = give_reg_model_rf()
model, rsqr  = test_model(model,xtrain,ytrain,xtest,ytest)

Dimension of Our Dataset:  (5926, 15421)
Random Forest
MSE:  0.0298
R-Squared :  0.8305
Margin  0.10  0.20  0.30  0.40  0.50
        45.11  76.64  92.16  97.47  99.33
 


In [331]:
import operator
sorted_d = sorted(cv.vocabulary_.items(), key=operator.itemgetter(1))
myvocab = [x[0] for x in sorted_d]
fnames = ['NE-Avg','%1 star','%2 star','%3 star','%4 star','%5 star'] + ["bucket"+str(1+(0.1*x)) for x in range(0,41)] + myvocab

In [341]:
feature_importances = {feature: importance for feature, importance in zip(fnames, list(model.feature_importances_))}
sorted_fi = sorted(feature_importances.items(), key=operator.itemgetter(1), reverse=True)
sorted_fi[:10]

[('NE-Avg', 0.46039427496662),
 ('%5 star', 0.28556612556360667),
 ('ordered', 0.004037451983372462),
 ('%4 star', 0.0030214840572476537),
 ('%3 star', 0.0029700939423346843),
 ('good', 0.0025148325479667096),
 ('decent', 0.0024646090118006607),
 ('bad', 0.0015549979307181885),
 ('tasted', 0.001531190090024915),
 ('place', 0.0014121830093210752)]

In [339]:
%matplotlib qt
plt.xticks(rotation='vertical')
plt.bar([a[0] for a in sorted_fi[:10]],[a[1] for a in sorted_fi[:10]])

<BarContainer object of 10 artists>